In [1]:
from PIL import Image
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import clip
import torch
from utils import ModelWrapper
import glob
import matplotlib.pyplot as plt

In [2]:
class_names = ['Amiodarone','Dexamethasone', 'Etomidate', 'Fentanyl', 'Glycopyrrolate', 
'Hydromorphone','Ketamine', 'Ketorolac', 'Lidocaine', 'Magnesium Sulfate', 
'Midazolam', 'Nesostigmine', 'Ondansetron', 'Propofol', 'Rocuronium', 'Vasopressin','Vecuronium']

In [3]:
if __name__ == '__main__':
    base_model, preprocess = clip.load('ViT-L/14', 'cuda', jit=False)
    NUM_CLASSES = 17
    model = ModelWrapper(base_model, base_model.visual.output_dim, NUM_CLASSES, normalize=True)
    for p in model.parameters():
        p.data = p.data.float()

    # TODO change to where model is saved
    
    saved_model_path = '/home/..../finetune_cp_4.pt'
    model.load_state_dict(torch.load(saved_model_path, map_location='cpu'))
    for p in model.parameters():
        p.data = p.data.float()

    model = model.cuda()
    devices = [x for x in range(torch.cuda.device_count())]
    model = torch.nn.DataParallel(model,  device_ids=devices)

In [4]:
import decimal
cls = 'Midazolam'
class_correct = []
class_wrong = []

# TODO change to where the Vial Label crops are saved
for image in glob.glob('/home/...../yolov5/runs/detect/exp/crops/Vial Label/*.jpg'):
    img = preprocess(Image.open(image)).unsqueeze(0)
    img = img.cuda()
    out = model(img)

      
    prediction = out.argmax(dim=1, keepdim=True)
    pred = class_names[prediction.item()]
    probabilities = torch.nn.functional.softmax(out, dim=1)
    top_p, top_class = probabilities.topk(5,1,largest=True,sorted=True)
    


    if pred == cls:
        class_correct.append(1)
    else:
        class_wrong.append(1)
        print(image)
        print(pred)
print(len(class_correct))
print(len(class_wrong))

/home/azureuser/cloudfiles/code/Users/sn74/drug_classifier/video_vial_label_data/Vial_Solomon_Clip_20/Vial_Solomon_Clip_20155.jpg
Dexamethasone
tensor([[0.5414, 0.2441, 0.1601, 0.0175, 0.0174]], device='cuda:0',
       grad_fn=<TopkBackward0>)
tensor([[ 1, 11,  0, 12,  8]], device='cuda:0')
/home/azureuser/cloudfiles/code/Users/sn74/drug_classifier/video_vial_label_data/Vial_Solomon_Clip_20/Vial_Solomon_Clip_20156.jpg
Nesostigmine
tensor([[0.5766, 0.3862, 0.0177, 0.0069, 0.0043]], device='cuda:0',
       grad_fn=<TopkBackward0>)
tensor([[11,  1,  0, 12,  8]], device='cuda:0')
/home/azureuser/cloudfiles/code/Users/sn74/drug_classifier/video_vial_label_data/Vial_Solomon_Clip_20/Vial_Solomon_Clip_20157.jpg
Dexamethasone
tensor([[0.9129, 0.0750, 0.0069, 0.0022, 0.0009]], device='cuda:0',
       grad_fn=<TopkBackward0>)
tensor([[ 1, 11, 12, 10,  8]], device='cuda:0')
/home/azureuser/cloudfiles/code/Users/sn74/drug_classifier/video_vial_label_data/Vial_Solomon_Clip_20/Vial_Solomon_Clip_20158